In [4]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model  # type: ignore
import pyttsx3

# Initialize the text-to-speech engine
engine = pyttsx3.init()

model = load_model('Actions.h5')  # Update this with the correct path

# Actions that we try to detect
actions = np.array(['hello', 'thank you', 'i love you'])

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Colors for actions
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


#they extract all the needed key points

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])




# Text-to-speech setup
last_spoken_action = None  # To avoid repeating the same action

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            # 3. Viz logic
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            # Text-to-Speech output
                            if actions[np.argmax(res)] != last_spoken_action:
                                last_spoken_action = actions[np.argmax(res)]
                                engine.say(last_spoken_action)
                                engine.runAndWait()
                    else:
                        sentence.append(actions[np.argmax(res)])
                        # Text-to-Speech output
                        if actions[np.argmax(res)] != last_spoken_action:
                            last_spoken_action = actions[np.argmax(res)]
                            engine.say(last_spoken_action)
                            engine.runAndWait()

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors) ----edited
        
        #adding height and width
        height, width, _ = image.shape

        # Display predicted action and confidence
        if len(predictions) > 0:
            action = actions[np.argmax(res)]
            confidence = res[np.argmax(res)]
            cv2.putText(image, f'{action} ({confidence:.2f})', (10, height - 10), 
                        cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show the last 5 detected actions
        cv2.rectangle(image, (0, 0), (640, 40), (0, 0, 0), -1)  # Background for text
        cv2.putText(image, ' '.join(sentence), (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show the OpenCV feed
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
hello
1

In [6]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import pyttsx3
import threading
import tkinter as tk

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Load the pre-trained model
model = load_model('Actions.h5')  # Update this with the correct path

# Actions that the model can detect
actions = np.array(['hello', 'thank you', 'i like you'])

# Variables for detection
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Text-to-speech setup
last_spoken_action = None  # To avoid repeating the same action

# Initialize flags for controlling the webcam thread
running = False
cap = None

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                   # Image is no longer writeable
    results = model.process(image)                  # Make prediction
    image.flags.writeable = True                    # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

def extract_keypoints(results):
    # Extract keypoints from results
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

def start_recognition():
    global running, cap
    if not running:
        running = True
        cap = cv2.VideoCapture(0)
        threading.Thread(target=run_recognition, daemon=True).start()

def stop_recognition():
    global running
    running = False

def run_recognition():
    global running, cap, sequence, sentence, predictions, last_spoken_action
    cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)  # Allow the window to be resizable
    cv2.resizeWindow('OpenCV Feed', 720, 480)  # Set initial size of the window
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while running:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            
            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                # Visualization logic
                if np.unique(predictions[-10:])[0] == np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 
                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                                # Text-to-Speech output
                                if actions[np.argmax(res)] != last_spoken_action:
                                    last_spoken_action = actions[np.argmax(res)]
                                    engine.say(last_spoken_action)
                                    engine.runAndWait()
                        else:
                            sentence.append(actions[np.argmax(res)])
                            # Text-to-Speech output
                            if actions[np.argmax(res)] != last_spoken_action:
                                last_spoken_action = actions[np.argmax(res)]
                                engine.say(last_spoken_action)
                                engine.runAndWait()

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

            # Display predicted action and confidence
            height, width, _ = image.shape
            if len(predictions) > 0:
                action = actions[np.argmax(res)]
                confidence = res[np.argmax(res)]
                cv2.putText(image, f'{action} ({confidence:.2f})', (10, height - 10), 
                            cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show the last 5 detected actions
            cv2.rectangle(image, (0, 0), (640, 40), (0, 0, 0), -1)  # Background for text
            cv2.putText(image, ' '.join(sentence), (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show the OpenCV feed
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows()

# Initialize the Tkinter window
window = tk.Tk()
window.title("Sign Language Recognition")
window.geometry("720x480")
window.config(bg="black")

#adding text for heading
label_text= tk.Label(window, text="",)

# Add buttons for controlling the application
start_button = tk.Button(window, text="Start Recognition", command=start_recognition)
start_button.pack(pady=100)

stop_button = tk.Button(window, text="Stop Recognition", command=stop_recognition)
stop_button.pack(pady=10)

# Start the Tkinter event loop
window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30m